In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
241,AFG,Asia,Afghanistan,2020-08-28,38129.0,3.0,39.000,1401.0,0.0,2.286,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
414,ALB,Europe,Albania,2020-08-28,9083.0,156.0,159.429,266.0,3.0,4.000,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
656,DZA,Africa,Algeria,2020-08-28,43016.0,397.0,394.000,1475.0,10.0,9.143,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-27,France,256210.0,0.0,30579.0,0.0,0.0,0.0
0,2020-08-27,China,89819.0,0.0,4719.0,0.0,0.0,0.0
0,2020-08-27,Italy,263353.0,0.0,35482.0,0.0,0.0,0.0
0,2020-08-27,Spain,426001.0,0.0,29022.0,0.0,0.0,0.0
0,2020-08-27,United States,5906961.0,0.0,180936.0,0.0,0.0,0.0
0,2020-08-27,World,24680823.0,0.0,833773.0,0.0,0.0,0.0
0,2020-08-27,United Kingdom,330141.0,0.0,41514.0,0.0,0.0,0.0
0,2020-08-27,Germany,239340.0,0.0,9294.0,0.0,0.0,0.0
0,2020-08-27,Iran,367171.0,0.0,21148.0,0.0,0.0,0.0
0,2020-08-27,Turkey,263776.0,0.0,6204.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-28,France,256210.0,259698.0,30579.0,30576.0,-3488.0,3.0
0,2020-08-28,China,89819.0,89814.0,4719.0,4715.0,5.0,4.0
0,2020-08-28,Italy,263353.0,263949.0,35482.0,35463.0,-596.0,19.0
0,2020-08-27,Spain,426001.0,429507.0,29022.0,28996.0,-3506.0,26.0
0,2020-08-28,United States,5906961.0,5867785.0,180936.0,180824.0,39176.0,112.0
0,2020-08-28,World,24680823.0,24473843.0,833773.0,832002.0,206980.0,1771.0
0,2020-08-28,United Kingdom,330141.0,330368.0,41514.0,41477.0,-227.0,37.0
0,2020-08-28,Germany,239340.0,239507.0,9294.0,9288.0,-167.0,6.0
0,2020-08-28,Iran,367171.0,367796.0,21148.0,21137.0,-625.0,11.0
0,2020-08-28,Turkey,263776.0,263998.0,6204.0,6209.0,-222.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")